<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js005_wavePacketQED2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### html + css * js in the code cell

In [ ]:
#@title js005_wavePacketQED2D

%%html

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js005_wavePacketQED2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js005_wavePacketQED2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.15 created, last updated on 2018.11.11
//    ver 0.0.1  2019.01.09 v1, last updated on 2021.04.24
//    ver 0.0.2  2021.10.27 v2, last updated on 2021.10.27
//    ver 0.0.3  2023.02.12 v3, last updated on 2023.08.11
//
//
// --------------------  QED: Quantum Electron Dynamics 2D
//
// - time dependent Schrodinger equation: i(d/dt)psi(r,t) = H psi(r,t)
// - time evolution
//    psi(r,t+dt) = exp(-i dt H) psi(r,t),  (H:Hamiltonian of the system)
//      H = -delta/2 + V(r), delta = d^2/dx^2 + d^2/dy^2 + d^2/dz^2
//    psi(r,t+dt) = exp(-i dt H) psi(r,t) nearly=
//      {exp(-i(dt/2)V} {exp(i dt(delta/2)} {exp(-i(dt/2)V} psi(r,t)
// - algorism: {exp(i dt(delta/2)}
//     QED: Watanabe's algorism (semi-implicit method)
//     Naoki Watanabe, Masaru Tsukada; arXiv:physics/0011068v1
//     (Published from Physical Review E. 62, 2914, (2000).)
//
//     Cayley's form : exp(i dt delta/2) nearly= (1 + i dt delta/4)/(1 - i dt delta/4)
//       psi(r,t+dt) = exp(i dt delta/2) psi(r,dt)
//       (1 - i dt delta/4) psi(r,t+dt) = (1 + i dt delta/4) psi(r,t)
//
//     difference form psi(r,t) --> psi(j,n)
//        psi(j,n+1) - i (dt/dx^2)/4 {psi(j-1,n+1))-2psi(j,n+1)+psi(j+1,n+1)}
//               = psi(j,n) + i (dt/dx^2)/4 {psi(j-1,n))-2psi(j,n)+psi(j+1,n)}
//        x i(4dx^2/dt) by each term
//        psi(j-1,n+1) + A Psi(j,n+1) + psi(j+1,n+1) = -psi(j-1,n) + B Psi(j,n) -psi(j+1,n)
//          where A=(i4dx^2/dt)-2, B=(i4dx^2/dt)+2
//          bnj = -psi(j-1,n) + B Psi(j,n) -psi(j+1,n) is calculated using known psi(j,n)
//          psi(j-1,n+1) + A Psi(j,n+1) + psi(j+1,n+1) = bnj
//
//     solve tri-diagonal equation  A X = B
//         | a1  1  0  0 |  | x1 |    | b1 |
//         |  1 a2  1  0 |  | x2 |  = | b2 |
//         |  0  1 a3  1 |  | x3 |    | b3 |
//         |  0  0  1 a4 |  | x4 |    | b4 |
//
//       u(1) = 1.0/a(1)  // u() : work vector
//       x(1) = b(1)*u(1)
//
//       for(i=2; i<=N-2; i++) { //forward elimination
//          u[i] = 1/(a[i]-u[i-1])
//          x[i] = (b[i]-x[i-1])*u[i]
//       }
//       for(i=N-3; i<=1; i--) { //backward substitution
//          x[i] -= x[i+1]*u[i]
//       }
//
// --------------------
*/

const wavePacketQED2D = (function(){ // ====================  wavePacketQED2D Module  ====================

	// au: atomic unit hBar=1,e=1,me=1,a0=1
	const g_auLength = 5.29177211e-11;				// (m) 1(au) = auLength (m)
	const g_auTime = 2.418884326e-17;				// (s) 1(au) = auTime (s)
	const g_auEnergy = 4.35974465e-18;				// (J) 1(au) = auEnergy (J)
	const g_au2eV = 27.211386;						// (eV) 1(au) = 27.211386 (eV)
	const g_Pi = Math.PI;							// Pi = 3.141592653589792...
	const g_nxMax = 320;							// NNx max
	const g_nyMax = 320;							// NNy max
	const g_nMax = 320;								// = Math.max(nxMax,nyMax);

	let g_NNx = 160;								// number of space x-division, NNx*dx == x-boxsize
	let g_NNy = 160;								// number of space y-division, NNy*dy == y-boxsize
	let g_sysTime = 0.0;							// (au) system time
	let g_dx = 0.5;									// (au) x-division
	let g_dy = 0.5;									// (au) y-division
	let g_timeStep = 0.5*g_dx*g_dx;					// (au) time step dt=0.5*dx*dx
	let g_dampingFactor = 0.05;						// for lossEnergy() :steepest descent method

	const g_psi = dim3( g_nxMax, g_nyMax, 2 );		// psi[nMax][nMax][2] wave function
													// [][][0]:real part [][][1]:imaginary part
	const g_vv = dim2( g_nxMax, g_nyMax );			// vv[nMax][nMax] external potential
	const g_wrk = dim3( g_nxMax, g_nyMax, 2 );		// wrk[nMax][nMax][2] work space in lossEnergy()
	const g_bRe = dim1( g_nMax );					// real part of b vector in kxStep(),kyStep()
	const g_bIm = dim1( g_nMax );					// imaginal part of b vector in kxStep(),kyStep()
	const g_uRe = dim1( g_nMax );					// real part of u vector in kxStep(),kyStep()
	const g_uIm = dim1( g_nMax );					// imaginal part of u vector in kxStep(),kyStep()
	const g_srnd = dim1( 1002 );					// 1000 RND orderd series 0 to 1,use drawCloud()
	const g_cloud = dimInt2( g_nxMax, g_nyMax );	// if cloud[i][j]>0, plot cloud point

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition(theme,canvasId) {
		// theme  0:free space, 1:parabolic potential, 2:tunnel effect, 3:step hill,
		//        4:double slit, 5:cylinder scattering
		const xMax=g_NNx*g_dx, yMax=g_NNy*g_dy, dx=g_dx, dy=g_dy;

		g_sysTime = 0.0;
		if (theme==0) { // 0:free space
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setWallPotential(0.5*xMax, 5*dx, 0.0); //(xPos, vThick, vHeight)
		} else if (theme==1) { // 1:parabolic potential
			setGaussianWave(xMax/2.0,yMax/4.0,3.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setParabolicPotential(2.0); // v(x)=k0*(x-x0)^2, k0=2.0
		} else if (theme==2) { // 2:tunnel effect
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky) K=0.5
			setWallPotential(0.5*xMax, 5*dx, 0.55); //(xPos, vThick, vHeight)
		} else if (theme==3) { // 3:step hill
			setGaussianWave(0.25*xMax,0.25*yMax,5.0,1.0,1.0); //(xPos,yPos,waveWidth,kx,ky)
			setWallPotential(0.5*xMax, xMax, 0.4); //(xPos, vThick, vHeight)
		} else if (theme==4) { // 4:double slit
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setSlitPotential(0.5*xMax,5*dx,4.0,0.05*yMax,0.10*yMax); //(Pos,Thick,Height,slitWidth,slitSpan)
		} else if (theme==5) { // 5:cylinder
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setCylinderPotential(0.6*xMax,0.4*yMax,0.1*xMax,1.0); //(xPos,yPos, radius, vHeight)
		}
	}

	function setGaussianWave(xPos,yPos,waveWidth,kx,ky) {
		const nnx=g_NNx, nny=g_NNy, dx=g_dx, dy=g_dy;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const x = i*dx;
				const y = j*dy;
				const phAb = Math.exp(-((x-xPos)*(x-xPos)+(y-yPos)*(y-yPos))/(4*waveWidth*waveWidth) );
				const phPh = kx*x+ky*y;
				g_psi[i][j][0] = phAb*Math.cos(phPh);
				g_psi[i][j][1] = phAb*Math.sin(phPh);
			}
		}
		for (let i=0; i<nnx; i++) {
			g_psi[i][0][0] = 0.0;
			g_psi[i][0][1] = 0.0;
			g_psi[i][nny-1][0] = 0.0;
			g_psi[i][nny-1][1] = 0.0;
		}
		for (let j=0; j<nny; j++) {
			g_psi[0][j][0] = 0.0;
			g_psi[0][j][1] = 0.0;
			g_psi[nnx-1][j][0] = 0.0;
			g_psi[nnx-1][j][1] = 0.0;
		}
		normalize(g_psi);
	}

	function setParabolicPotential(k0) { // V(r)= k0*r^2
		const nnx=g_NNx, nny=g_NNy, dx=g_dx, dy=g_dy;

		const aa = k0/(nnx*dx*nnx*dx/4.0);
		const x0 = nnx*dx/2.0;
		const y0 = nny*dy/2.0;
		for (let i=0; i<nnx; i++) {
			const x = i*dx
			for (let j=0; j<nny; j++) {
				const y = j*dy;
				g_vv[i][j] = aa*((x-x0)*(x-x0)+(y-y0)*(y-y0));
			}
		}
	}

	function setWallPotential(xPos, vThick, vHeight) {
		const nnx=g_NNx, nny=g_NNy, dx=g_dx, dy=g_dy;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const x = i*dx;
				const y = j*dy;
				g_vv[i][j] = (x>=xPos && x<xPos+vThick) ? vHeight : 0.0;
			}
		}
	}

	function setCylinderPotential(xPos,yPos, radius, vHeight) {
		const nnx=g_NNx, nny=g_NNy, dx=g_dx, dy=g_dy;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const x = i*dx-xPos;
				const y = j*dy-yPos;
				g_vv[i][j] = (x*x+y*y < radius*radius) ? vHeight : 0.0;

			}
		}
	}

	function setSlitPotential(wallPos, wallThick, wallHeight, slitWidth, slitSpan) {
		const nnx=g_NNx, nny=g_NNy, dx=g_dx, dy=g_dy;

		const ym = nny*dy/2.0;
		const w = slitWidth/2.0;
		const d = slitSpan/2.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const x = i*dx;
				const y = j*dy;
				if (x>=wallPos && x<wallPos+wallThick) {
					g_vv[i][j] = wallHeight;
					if ((y>=ym-d-w && y<=ym-d+w) || (y>=ym+d-w && y<=ym+d+w)) g_vv[i][j] = 0.0;
				} else {
					g_vv[i][j] = 0.0;
				}
			}
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( lossSW ) {
		// lossSW   0:no loss, 1:loss

		for (let i=0; i<2; i++) {
			g_sysTime += g_timeStep;
			phaseStep(g_psi,g_vv,0.5*g_timeStep);
			kxStep(g_psi,g_timeStep);
			kyStep(g_psi,g_timeStep);
			phaseStep(g_psi,g_vv,0.5*g_timeStep);
		}
		if (lossSW==1) lossEnergy(g_psi,g_vv,g_dampingFactor);
	}

	function phaseStep( ph, vv, dt ) {
		const nnx=g_NNx, nny=g_NNy;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const th = dt*g_vv[i][j];
				const cs = Math.cos(th);
				const sn = Math.sin(th);
				const phr = ph[i][j][0];
				const phi = ph[i][j][1];
				ph[i][j][0] = cs*phr+sn*phi;
				ph[i][j][1] = cs*phi-sn*phr;
			}
		}
	}

	function kxStep( ph, dt ) {
		const nnx=g_NNx, nny=g_NNy;

		const a = 4.0*g_dy*g_dy/dt;
		const aaAb = 4.0+a*a;
		for (let j=1; j<nny-1; j++) {
			// set b[], u[1],ph[1][]
			for (let i=1; i<nnx-1; i++) {
				g_bRe[i] = 2.0*ph[i][j][0]-a*ph[i][j][1] - ph[i+1][j][0] - ph[i-1][j][0];
				g_bIm[i] = 2.0*ph[i][j][1]+a*ph[i][j][0] - ph[i+1][j][1] - ph[i-1][j][1];
			}
			g_uRe[1] = -2.0/aaAb;
			g_uIm[1] = -a/aaAb;
			ph[1][j][0] = g_bRe[1]*g_uRe[1] - g_bIm[1]*g_uIm[1];
			ph[1][j][1] = g_bIm[1]*g_uRe[1] + g_bRe[1]*g_uIm[1];
			//forward elimination
			for (let i=2; i<nnx-1; i++) { //forward elimination
				const auAb = (-2.0-g_uRe[i-1])*(-2.0-g_uRe[i-1])+(a-g_uIm[i-1])*(a-g_uIm[i-1]);
				g_uRe[i] = (-2.0-g_uRe[i-1])/auAb;
				g_uIm[i] = -(a-g_uIm[i-1])/auAb;
				ph[i][j][0] = (g_bRe[i]-ph[i-1][j][0])*g_uRe[i] - (g_bIm[i]-ph[i-1][j][1])*g_uIm[i];
				ph[i][j][1] = (g_bRe[i]-ph[i-1][j][0])*g_uIm[i] + (g_bIm[i]-ph[i-1][j][1])*g_uRe[i];
			}
			//backward substitution
			for (let i=nnx-3; i>=1; i--) {
				ph[i][j][0] -= ph[i+1][j][0]*g_uRe[i] - ph[i+1][j][1]*g_uIm[i];
				ph[i][j][1] -= ph[i+1][j][0]*g_uIm[i] + ph[i+1][j][1]*g_uRe[i];
			}
		}
	}

	function kyStep( ph, dt ) {
		const nnx=g_NNx, nny=g_NNy;

		const a = 4.0*g_dy*g_dy/dt;
		const aaAb = 4.0+a*a;
		for (let i=1; i<nnx-1; i++) {
			// set b[], u[1],ph[][1]
			for (let j=1; j<nny-1; j++) {
				g_bRe[j] = 2.0*ph[i][j][0]-a*ph[i][j][1] - ph[i][j+1][0] - ph[i][j-1][0];
				g_bIm[j] = 2.0*ph[i][j][1]+a*ph[i][j][0] - ph[i][j+1][1] - ph[i][j-1][1];
			}
			g_uRe[1] = -2.0/aaAb;
			g_uIm[1] = -a/aaAb;
			ph[i][1][0] = g_bRe[1]*g_uRe[1] - g_bIm[1]*g_uIm[1];
			ph[i][1][1] = g_bIm[1]*g_uRe[1] + g_bRe[1]*g_uIm[1];
			//forward elimination
			for (let j=2; j<nny-1; j++) {
				const auAb = (-2.0-g_uRe[j-1])*(-2.0-g_uRe[j-1])+(a-g_uIm[j-1])*(a-g_uIm[j-1]);
				g_uRe[j] = (-2.0-g_uRe[j-1])/auAb;
				g_uIm[j] = -(a-g_uIm[j-1])/auAb;
				ph[i][j][0] = (g_bRe[j]-ph[i][j-1][0])*g_uRe[j] - (g_bIm[j]-ph[i][j-1][1])*g_uIm[j];
				ph[i][j][1] = (g_bRe[j]-ph[i][j-1][0])*g_uIm[j] + (g_bIm[j]-ph[i][j-1][1])*g_uRe[j];
			}
			//backward substitution
			for (let j=nny-3; j>=1; j--) {
				ph[i][j][0] -= ph[i][j+1][0]*g_uRe[j] - ph[i][j+1][1]*g_uIm[j];
				ph[i][j][1] -= ph[i][j+1][0]*g_uIm[j] + ph[i][j+1][1]*g_uRe[j];
			}
		}
	}

	// steepest descent method: psi_next = |psi) - damp*|psi)(psi|H-E|psi)
	// H = d^2/dx^2 + d^2/dy^2 + V
	function lossEnergy( ph, vv, damp ) {
		const nnx=g_NNx, nny=g_NNy, h2 = 2.0*g_dx*g_dx, ee = meanKinetic(ph) + meanPotential(ph,vv);

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_wrk[i][j][0] = -(ph[i+1][j][0]+ph[i-1][j][0]+ph[i][j+1][0]+ph[i][j-1][0]
								-4.0*ph[i][j][0])/h2+(g_vv[i][j]-ee)*ph[i][j][0];
				g_wrk[i][j][1] = -(ph[i+1][j][1]+ph[i-1][j][1]+ph[i][j+1][1]+ph[i][j-1][1]
								-4.0*ph[i][j][1])/h2+(g_vv[i][j]-ee)*ph[i][j][1];
			}
		}
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				ph[i][j][0] -= damp*g_wrk[i][j][0];
				ph[i][j][1] -= damp*g_wrk[i][j][1];
			}
		}
		normalize(ph);
	}


	// --------------------  utility  --------------------

	function norm( ph ) {
		const nnx=g_NNx, nny=g_NNy;

		let p = 0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				p += (ph[i][j][0]*ph[i][j][0]+ph[i][j][1]*ph[i][j][1]);
			}
		}
		return p*g_dx*g_dy;
	}

	function normalize( ph ) {
		const nnx=g_NNx, nny=g_NNy;

		const a = 1.0/Math.sqrt(norm(ph));
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				ph[i][j][0] *= a;
				ph[i][j][1] *= a;
			}
		}
	}

	function meanPotential( ph, vv ) {
		const nnx=g_NNx, nny=g_NNy;

		let p = 0.0
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				p += vv[i][j]*(ph[i][j][0]*ph[i][j][0]+ph[i][j][1]*ph[i][j][1]);
			}
		}
		return p*g_dx*g_dy;
	}

	function meanKinetic( ph ) {
		const nnx=g_NNx, nny=g_NNy, h2=g_dx*g_dx;
		let p = 0.0;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const d2phRe = (ph[i+1][j][0]+ph[i-1][j][0]+ph[i][j+1][0]+ph[i][j-1][0]-4.0*ph[i][j][0])/h2;
				const d2phIm = (ph[i+1][j][1]+ph[i-1][j][1]+ph[i][j+1][1]+ph[i][j-1][1]-4.0*ph[i][j][1])/h2;
				p += (ph[i][j][0]*d2phRe+ph[i][j][1]*d2phIm);
			}
		}
		return -0.5*p*g_dx*g_dy;
	}

	function psiDensity(i,j) {
		return (g_psi[i][j][0]*g_psi[i][j][0]+g_psi[i][j][1]*g_psi[i][j][1]);
	}

	function psiPhase(i,j) { // 0...2PI
		return (Math.PI + Math.atan2(g_psi[i][j][1],g_psi[i][j][0])); // atan2(y,x)
	}

	function psiXCurrent(i,j) {
		const pRe = (g_psi[i+1][j][1]-g_psi[i-1][j][1])/(2*g_dx);
		const pIm = (-g_psi[i+1][j][0]+g_psi[i-1][j][0])/(2*g_dx);

		return (g_psi[i][j][0]*pRe + g_psi[i][j][1]*pIm)*g_dx*g_dy;
	}

	function psiYCurrent(i,j) {
		const pRe = (g_psi[i][j+1][1]-g_psi[i][j-1][1])/(2*g_dy);
		const pIm = (-g_psi[i][j+1][0]+g_psi[i][j-1][0])/(2*g_dy);

		return (g_psi[i][j][0]*pRe + g_psi[i][j][1]*pIm)*g_dx*g_dy;
	}

	function setCloud() {
		const nnx=g_NNx, nny=g_NNy;

		set_srnd();

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_cloud[i][j] = 0;
			}
		}
		let s = 0;
		let ip = 0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_cloud[i][j] = 0;
				s += psiDensity(i,j)*g_dx*g_dy;
				while (s>g_srnd[ip] && ip<1000) {
					g_cloud[i][j] += 1;
					ip += 1;
				}
			}
		}
	}

	function set_srnd() {

		g_srnd[0] = Math.random();
		for (let i=1; i<1001; i++) {
			g_srnd[i] = g_srnd[i-1] + Math.random();
		}
		for (let i=0; i<1000; i++) {
			g_srnd[i] = g_srnd[i]/g_srnd[1000];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme )
		evolve:			timeEvolution,			// timeEvolution( lossSW )

		setCloud:		setCloud,				// setCloud()

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_NNy, g_dx, g_timeStep ]; },
		getNow:			function() { return [ g_sysTime, norm(g_psi), meanKinetic(g_psi), meanPotential(g_psi,g_vv) ]; },
		getPsiDensity:	psiDensity,				// psiDensity(i,j)
		getPsiPhase:	psiPhase,				// psiPhase(i,j)
		getPsiCurrent:	function(i,j) { return [ psiXCurrent(i,j), psiYCurrent(i,j) ]; },
		getPsi:			function(i,j) { return [ g_psi[i][j][0], g_psi[i][j][1] ]; },
		getVext:		function(i,j) { return g_vv[i][j]; },
		getCloud:		function(i,j) { return g_cloud[i][j]; },
	};

})(); // ====================  end of wavePacketQED2D Module  ====================


const js005 = (function(){ // ====================  js Module  ====================

	const theModule = wavePacketQED2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');
	let imgVext = null;			// image in draw()

	let v_theme = 1;			// 0:free space, 1:parabolic potential, 2:tunnel effect 3:step hill 4:step higher hill
	let v_lossSW = 0;			// if lossSW==1, call lossEnergy()

	let p_auLength, p_auTime, p_auEnergy, p_au2eV; // <-- theModule.getAUinSI();
	let p_NNx, p_NNy, p_dx, p_timeStep; // <-- theModule.getSysParam();
	let p_NNz;
	let sysTime, norm, meanKinetic, meanPotential;

	let dispMode = 1;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let onlyFlag = false;
  let breakFlag = false;

	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics - mouse event listener
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme ); // ( nn, BoxSizeInNM, contTemp )
			imgVext = null;
			[ p_auLength, p_auTime, p_auEnergy, p_au2eV ] = theModule.getAUinSI();
			[ p_NNx, p_NNy, p_dx, p_timeStep ] = theModule.getSysParam();
			p_NNz = p_NNx;
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift )
			g3d.init( p_NNx, p_NNy, p_NNz, p_dx, xCanvasSize, yCanvasSize, 300, 20 );
			g3d.drawVectorField2D.threshold = 0.2;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_lossSW );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			onlyFlag = true;
			theModule.evolve( v_lossSW );
		}

		draw( ctx, dispMode );

		requestAnimationFrame(animate);
	}


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const xp = 80, yp = 20, sc = 2, xBoxSize = 320, yBoxSise = 320, yTextPos = yCanvasSize-45; // image2d
		const colour = { text:'#888888', box:'#999900', vext:'#00ff00' };

		ctx.clearRect( 0, 0, xCanvasSize, yCanvasSize );

		if ( dispMode<=3 ) {
			ctx.strokeStyle = colour.box;
			ctx.strokeRect( xp, yp, xBoxSize, yBoxSise );

			if ( imgVext == null ) setImg_Vext( ctx, p_NNx, p_NNy, xp, yp, sc );
			ctx.putImageData( imgVext, xp, yp ); // potential vv[][] imageData
		}

		if ( dispMode==0 || dispMode==1 || dispMode==2 ) { // 0:density 1:phase 2::prob.current
			drawPh( ctx, dispMode, p_NNx, p_NNy, xp, yp, sc );
		} else if ( dispMode==3 ) { // 3:cloud
			drawCloud( ctx, p_NNx, p_NNy, xp, yp, sc );

		} else if (dispMode==4 ) { // 4: 3D Vext(x,y)
			const zFunc = function(i,j) { return 10.0*theModule.getVext(i,j); };
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, "#00aa00", 2 );
		} else if (dispMode==5 ) { // 4: 3D density(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j); };
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, 1.0, 2 );
		} else if (dispMode==6 ) { // 4: 3D Re[psi](x,y)
			const zFunc = function(i,j) { return 150.0*theModule.getPsi(i,j)[0]; };
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, 1.0, 1 );
		} else if (dispMode==7 ) { // 7: 3D phase(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j)};
			const colorFunc = function(i,j) {
				const hue = theModule.getPsiPhase(i,j)*180.0/Math.PI;
				const z = 1500.0*theModule.getPsiDensity(i,j);
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? "hsl("+hue+",100%,50%)" : "#444444";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );
		} else if (dispMode==8 ) { // 8: 3D (density+Vext)(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j) + 10.0*theModule.getVext(i,j) };
			const colorFunc = function(i,j) {
				const zc = 1500.0*theModule.getPsiDensity(i,j)
				const th = (18120 - Math.floor(180.0*zc/g3d.cz0))%360;
				return ( Math.abs(zc/g3d.cz0)>0.005 ) ? "hsl("+(th)+",100%,50%)" : "hsl("+(th)+",30%,20%)";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );
		} else if (dispMode==9 ) { // 9: 3D (phase+Vext)(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j) + 10.0*theModule.getVext(i,j) };
			const colorFunc = function(i,j) {
				const hue = theModule.getPsiPhase(i,j)*180.0/Math.PI;
				const z = 1500.0*theModule.getPsiDensity(i,j);
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? "hsl("+(hue)+",100%,50%)" : "#004400";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );
		} else if (dispMode==10 ) { // 10: 3D (density+Vext)(x,y) + prob.current
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j) + 10.0*theModule.getVext(i,j) };
			const colorFunc = function(i,j) {
				const zc = 1500.0*theModule.getPsiDensity(i,j)
				const th = (18120 - Math.floor(180.0*zc/g3d.cz0))%360;
				return ( Math.abs(zc/g3d.cz0)>0.005 ) ? "hsl("+(th)+",100%,50%)" : "hsl("+(th)+",30%,20%)";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );

			const vFunc = function(i,j) {
				const mag = 15000.0;
				const z = 1500.0*theModule.getPsiDensity(i,j)
				if ( z<0.01 ) return [ 0, 0 ];
				let cx,cy;
				[ cx, cy ] = theModule.getPsiCurrent(i,j);
				return [ cx*mag, cy*mag ];
			}
			//drawVectorField2D(ctx, rotAngle, zFunc, vFunc, colorMode, inc [, showBox] )
			g3d.drawVectorField2D(ctx, 0.0, zFunc, vFunc, 1, 4, 2 )
		}

		[ sysTime, norm, meanKinetic, meanPotential ] = theModule.getNow();
		const totalEnergy = meanKinetic + meanPotential;
		ctx.fillStyle = colour.text;
		ctx.fillText(`time = ${sysTime} (au)`, 20, yTextPos);
		ctx.fillText(`norm = ${norm.toFixed(15)}`, 240, yTextPos);
		ctx.fillText(`kineticE = ${(meanKinetic*p_au2eV).toFixed(2)} (eV)`, 20, yTextPos+20);
		ctx.fillText(`potentialE = ${(meanPotential*p_au2eV).toFixed(2)} (eV)`, 240, yTextPos+20);
		ctx.fillText(`totalE= ${(totalEnergy*p_au2eV).toFixed(2)} (eV)`, 20, yTextPos+40);
		ctx.fillText("box = 80x80 (au)", 240, yTextPos+40);

	}

	function setImg_Vext( ctx, nnx, nny, xp, yp, sc ) {
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const vij = theModule.getVext(i,j);
				const g = Math.min(Math.floor(vij*50),255);
				ctx.fillStyle = "rgb(0,"+g+",0)";
				ctx.fillRect( i*sc+xp, (nny-j-1)*sc+yp, sc, sc );
			}
		}
		imgVext = ctx.getImageData(xp,yp,320,320);
	}

	function drawPh( ctx, drawMode, nnx, nny, xp, yp, sc ) {
		const velocityMag = 300000.0;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const d = theModule.getPsiDensity(i,j)*100000;
				const vij = theModule.getVext(i,j);
				const g = Math.min(Math.floor(vij*50),255);
				if (d>20.0) {
					if (drawMode==0 ||drawMode==2) { // 0:density 2:prob.current
						const a = Math.min(Math.floor(d),255);
						ctx.fillStyle = `rgb(${a},${g},${a})`;
					} else if (drawMode==1) { // 1:phase
						const p = theModule.getPsiPhase(i,j)*180.0/Math.PI;
						const a = 0.2*Math.min(Math.floor(d),255);
						ctx.fillStyle = `hsl(${p},100%,${a}%)`;
					}
					ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp,sc,sc);
				}
			}
		}
		if (drawMode==2) { // 2:prob.current, draw current lines
			for (let i=1; i<nnx-1; i+=4) {
				for (let j=1; j<nny-1; j+=4) {
					const d = theModule.getPsiDensity(i,j)*100000;
					if (d>20.0) {
						let cxij, cyij;
						[ cxij, cyij ] = theModule.getPsiCurrent(i,j)
						const cx = cxij*p_timeStep*velocityMag;
						const cy = -cyij*p_timeStep*velocityMag;
						const col = (cx>0) ? "rgb(80,80,250)" : "rgb(250,80,80)";
						drawLine( ctx, i*sc+xp, (nny-j-1)*sc+yp, i*sc+cx+xp, (nny-j-1)*sc+cy+yp, col );
					}
				}
			}
		}
	}

	function drawCloud( ctx, nnx, nny, xp, yp, sc ) { // drawMode = 3:cloud

		if ( !pauseFlag || onlyFlag ) {
			theModule.setCloud();
			onlyFlag = false;
		}
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const cij = theModule.getCloud(i,j)
				if ( cij>0 ) {
					const vij = theModule.getVext(i,j);
					const g = Math.min(Math.floor(vij*50),255);
					const p = Math.min( cij, 5 ) ;
					ctx.fillStyle = "hsl("+(75-p*15)+",100%,50%)";
					ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp,sc,sc);
				}
			}
		}
	}

	function drawLine( ctx, x1, y1, x2, y2, color ) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------

	// g3d_extension grid2d  created 2023.06.01, last updated 2023.06.04
	// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
	g3d.drawGrid2D = function ( ctx, rotAngle, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawGrid2D.threshold = 0.005;

	// g3d_extension gvextor field  created 2023.06.01, last updated 2023.06.08
	// g3d.drawVectorField2D(ctx, rotAngle, zFunc, vFunc, colorMode, inc [, showBox] )
	g3d.drawVectorField2D = function(ctx, rotAngle, zFunc, vFunc, colorMode, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, threshold = g3d.drawVectorField2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii; if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj+=inc) {
				let j=jj; if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;

				const f = zFunc(i,nny-j-1);
				const x = i*dx, y = j*dx, z = f + g3d.cz0;
				let vx, vy;
				[ vx, vy ] = vFunc(i,nny-j-1);
				if ( vx*vx+vy*vy < threshold ) continue;
				const x1 = x + vx, y1 = y -vy, z1 = z;
				let colr;
				if ( colorMode==0 ) {
					const hue = Math.floor((Math.atan2(vy,vx))*180.0/Math.PI+180.0);
					colr = `hsl(${hue},100%,50%)`;
				} else if ( colorMode==1 ) {
					colr = (vx>0) ? "#3333ff" : "#ff3333";
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawVectorField2D.threshold = 0.5;


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setLossSW() {  // select lossSW
		v_lossSW = 0 + document.getElementById("slct_loss").selectedIndex;
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle( 15, -10 );
		g3d.zoom = 1.0;
	}

	function breakLoop() {
		breakFlag = true;
	}


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setLossSW:		setLossSW,		// setLossSW()
		setDispMode:	setDispMode,	// setDispMode()
		viewHome:		viewHome,		// viewHome()

    breakLoop:	breakLoop,	// breakLoop();
	};

})(); // ====================  js005 module end  ====================


const js = js005;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js005] electron wave packet QED2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>in free box</option><option selected>in parabollic V=k r^2</option>
<option>tunnel effect</option><option>step hill</option>
<option>double slit</option><option>cylinder scattering</option>
</select>
    <span style="margin-right: 10px;"></span>
<label>loss:</label>
<select id="slct_loss" onChange="js.setLossSW()">
<option selected>off</option><option>on</option></select>
    <span style="margin-right: 20px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>2D: psi density</option><option selected>2D: phase (angle:Im(psi)/Re(psi))</option>
<option>2D: probability current</option><option>2D electron cloud</option>
<option>grid2D: Vext(x,y)</option><option>grid2D: density(x,y)</option>
<option>grid2D: Re[psi(x,y)]</option><option>grid2D: phase(x,y)</option>
<option>grid2D: (density + Vext)(x,y)</option><option>grid2D: (phase + Vext)(x,y)</option>
<option>grid2D: prob.current +(density + Vext)(x,y)</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
<br>

</body>
</html>


In [2]:
#@title def exec_html_js()

# def htm_ArMD2D

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js005_wavePacketQED2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js005_wavePacketQED2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.15 created, last updated on 2018.11.11
//    ver 0.0.1  2019.01.09 v1, last updated on 2021.04.24
//    ver 0.0.2  2021.10.27 v2, last updated on 2021.10.27
//    ver 0.0.3  2023.02.12 v3, last updated on 2023.08.11
//
//
// --------------------  QED: Quantum Electron Dynamics 2D
//
// - time dependent Schrodinger equation: i(d/dt)psi(r,t) = H psi(r,t)
// - time evolution
//    psi(r,t+dt) = exp(-i dt H) psi(r,t),  (H:Hamiltonian of the system)
//      H = -delta/2 + V(r), delta = d^2/dx^2 + d^2/dy^2 + d^2/dz^2
//    psi(r,t+dt) = exp(-i dt H) psi(r,t) nearly=
//      {exp(-i(dt/2)V} {exp(i dt(delta/2)} {exp(-i(dt/2)V} psi(r,t)
// - algorism: {exp(i dt(delta/2)}
//     QED: Watanabe's algorism (semi-implicit method)
//     Naoki Watanabe, Masaru Tsukada; arXiv:physics/0011068v1
//     (Published from Physical Review E. 62, 2914, (2000).)
//
//     Cayley's form : exp(i dt delta/2) nearly= (1 + i dt delta/4)/(1 - i dt delta/4)
//       psi(r,t+dt) = exp(i dt delta/2) psi(r,dt)
//       (1 - i dt delta/4) psi(r,t+dt) = (1 + i dt delta/4) psi(r,t)
//
//     difference form psi(r,t) --> psi(j,n)
//        psi(j,n+1) - i (dt/dx^2)/4 {psi(j-1,n+1))-2psi(j,n+1)+psi(j+1,n+1)}
//               = psi(j,n) + i (dt/dx^2)/4 {psi(j-1,n))-2psi(j,n)+psi(j+1,n)}
//        x i(4dx^2/dt) by each term
//        psi(j-1,n+1) + A Psi(j,n+1) + psi(j+1,n+1) = -psi(j-1,n) + B Psi(j,n) -psi(j+1,n)
//          where A=(i4dx^2/dt)-2, B=(i4dx^2/dt)+2
//          bnj = -psi(j-1,n) + B Psi(j,n) -psi(j+1,n) is calculated using known psi(j,n)
//          psi(j-1,n+1) + A Psi(j,n+1) + psi(j+1,n+1) = bnj
//
//     solve tri-diagonal equation  A X = B
//         | a1  1  0  0 |  | x1 |    | b1 |
//         |  1 a2  1  0 |  | x2 |  = | b2 |
//         |  0  1 a3  1 |  | x3 |    | b3 |
//         |  0  0  1 a4 |  | x4 |    | b4 |
//
//       u(1) = 1.0/a(1)  // u() : work vector
//       x(1) = b(1)*u(1)
//
//       for(i=2; i<=N-2; i++) { //forward elimination
//          u[i] = 1/(a[i]-u[i-1])
//          x[i] = (b[i]-x[i-1])*u[i]
//       }
//       for(i=N-3; i<=1; i--) { //backward substitution
//          x[i] -= x[i+1]*u[i]
//       }
//
// --------------------
*/

const wavePacketQED2D = (function(){ // ====================  wavePacketQED2D Module  ====================

	// au: atomic unit hBar=1,e=1,me=1,a0=1
	const g_auLength = 5.29177211e-11;				// (m) 1(au) = auLength (m)
	const g_auTime = 2.418884326e-17;				// (s) 1(au) = auTime (s)
	const g_auEnergy = 4.35974465e-18;				// (J) 1(au) = auEnergy (J)
	const g_au2eV = 27.211386;						// (eV) 1(au) = 27.211386 (eV)
	const g_Pi = Math.PI;							// Pi = 3.141592653589792...
	const g_nxMax = 320;							// NNx max
	const g_nyMax = 320;							// NNy max
	const g_nMax = 320;								// = Math.max(nxMax,nyMax);

	let g_NNx = 160;								// number of space x-division, NNx*dx == x-boxsize
	let g_NNy = 160;								// number of space y-division, NNy*dy == y-boxsize
	let g_sysTime = 0.0;							// (au) system time
	let g_dx = 0.5;									// (au) x-division
	let g_dy = 0.5;									// (au) y-division
	let g_timeStep = 0.5*g_dx*g_dx;					// (au) time step dt=0.5*dx*dx
	let g_dampingFactor = 0.05;						// for lossEnergy() :steepest descent method

	const g_psi = dim3( g_nxMax, g_nyMax, 2 );		// psi[nMax][nMax][2] wave function
													// [][][0]:real part [][][1]:imaginary part
	const g_vv = dim2( g_nxMax, g_nyMax );			// vv[nMax][nMax] external potential
	const g_wrk = dim3( g_nxMax, g_nyMax, 2 );		// wrk[nMax][nMax][2] work space in lossEnergy()
	const g_bRe = dim1( g_nMax );					// real part of b vector in kxStep(),kyStep()
	const g_bIm = dim1( g_nMax );					// imaginal part of b vector in kxStep(),kyStep()
	const g_uRe = dim1( g_nMax );					// real part of u vector in kxStep(),kyStep()
	const g_uIm = dim1( g_nMax );					// imaginal part of u vector in kxStep(),kyStep()
	const g_srnd = dim1( 1002 );					// 1000 RND orderd series 0 to 1,use drawCloud()
	const g_cloud = dimInt2( g_nxMax, g_nyMax );	// if cloud[i][j]>0, plot cloud point

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition(theme,canvasId) {
		// theme  0:free space, 1:parabolic potential, 2:tunnel effect, 3:step hill,
		//        4:double slit, 5:cylinder scattering
		const xMax=g_NNx*g_dx, yMax=g_NNy*g_dy, dx=g_dx, dy=g_dy;

		g_sysTime = 0.0;
		if (theme==0) { // 0:free space
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setWallPotential(0.5*xMax, 5*dx, 0.0); //(xPos, vThick, vHeight)
		} else if (theme==1) { // 1:parabolic potential
			setGaussianWave(xMax/2.0,yMax/4.0,3.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setParabolicPotential(2.0); // v(x)=k0*(x-x0)^2, k0=2.0
		} else if (theme==2) { // 2:tunnel effect
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky) K=0.5
			setWallPotential(0.5*xMax, 5*dx, 0.55); //(xPos, vThick, vHeight)
		} else if (theme==3) { // 3:step hill
			setGaussianWave(0.25*xMax,0.25*yMax,5.0,1.0,1.0); //(xPos,yPos,waveWidth,kx,ky)
			setWallPotential(0.5*xMax, xMax, 0.4); //(xPos, vThick, vHeight)
		} else if (theme==4) { // 4:double slit
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setSlitPotential(0.5*xMax,5*dx,4.0,0.05*yMax,0.10*yMax); //(Pos,Thick,Height,slitWidth,slitSpan)
		} else if (theme==5) { // 5:cylinder
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setCylinderPotential(0.6*xMax,0.4*yMax,0.1*xMax,1.0); //(xPos,yPos, radius, vHeight)
		}
	}

	function setGaussianWave(xPos,yPos,waveWidth,kx,ky) {
		const nnx=g_NNx, nny=g_NNy, dx=g_dx, dy=g_dy;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const x = i*dx;
				const y = j*dy;
				const phAb = Math.exp(-((x-xPos)*(x-xPos)+(y-yPos)*(y-yPos))/(4*waveWidth*waveWidth) );
				const phPh = kx*x+ky*y;
				g_psi[i][j][0] = phAb*Math.cos(phPh);
				g_psi[i][j][1] = phAb*Math.sin(phPh);
			}
		}
		for (let i=0; i<nnx; i++) {
			g_psi[i][0][0] = 0.0;
			g_psi[i][0][1] = 0.0;
			g_psi[i][nny-1][0] = 0.0;
			g_psi[i][nny-1][1] = 0.0;
		}
		for (let j=0; j<nny; j++) {
			g_psi[0][j][0] = 0.0;
			g_psi[0][j][1] = 0.0;
			g_psi[nnx-1][j][0] = 0.0;
			g_psi[nnx-1][j][1] = 0.0;
		}
		normalize(g_psi);
	}

	function setParabolicPotential(k0) { // V(r)= k0*r^2
		const nnx=g_NNx, nny=g_NNy, dx=g_dx, dy=g_dy;

		const aa = k0/(nnx*dx*nnx*dx/4.0);
		const x0 = nnx*dx/2.0;
		const y0 = nny*dy/2.0;
		for (let i=0; i<nnx; i++) {
			const x = i*dx
			for (let j=0; j<nny; j++) {
				const y = j*dy;
				g_vv[i][j] = aa*((x-x0)*(x-x0)+(y-y0)*(y-y0));
			}
		}
	}

	function setWallPotential(xPos, vThick, vHeight) {
		const nnx=g_NNx, nny=g_NNy, dx=g_dx, dy=g_dy;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const x = i*dx;
				const y = j*dy;
				g_vv[i][j] = (x>=xPos && x<xPos+vThick) ? vHeight : 0.0;
			}
		}
	}

	function setCylinderPotential(xPos,yPos, radius, vHeight) {
		const nnx=g_NNx, nny=g_NNy, dx=g_dx, dy=g_dy;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const x = i*dx-xPos;
				const y = j*dy-yPos;
				g_vv[i][j] = (x*x+y*y < radius*radius) ? vHeight : 0.0;

			}
		}
	}

	function setSlitPotential(wallPos, wallThick, wallHeight, slitWidth, slitSpan) {
		const nnx=g_NNx, nny=g_NNy, dx=g_dx, dy=g_dy;

		const ym = nny*dy/2.0;
		const w = slitWidth/2.0;
		const d = slitSpan/2.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const x = i*dx;
				const y = j*dy;
				if (x>=wallPos && x<wallPos+wallThick) {
					g_vv[i][j] = wallHeight;
					if ((y>=ym-d-w && y<=ym-d+w) || (y>=ym+d-w && y<=ym+d+w)) g_vv[i][j] = 0.0;
				} else {
					g_vv[i][j] = 0.0;
				}
			}
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( lossSW ) {
		// lossSW   0:no loss, 1:loss

		for (let i=0; i<2; i++) {
			g_sysTime += g_timeStep;
			phaseStep(g_psi,g_vv,0.5*g_timeStep);
			kxStep(g_psi,g_timeStep);
			kyStep(g_psi,g_timeStep);
			phaseStep(g_psi,g_vv,0.5*g_timeStep);
		}
		if (lossSW==1) lossEnergy(g_psi,g_vv,g_dampingFactor);
	}

	function phaseStep( ph, vv, dt ) {
		const nnx=g_NNx, nny=g_NNy;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const th = dt*g_vv[i][j];
				const cs = Math.cos(th);
				const sn = Math.sin(th);
				const phr = ph[i][j][0];
				const phi = ph[i][j][1];
				ph[i][j][0] = cs*phr+sn*phi;
				ph[i][j][1] = cs*phi-sn*phr;
			}
		}
	}

	function kxStep( ph, dt ) {
		const nnx=g_NNx, nny=g_NNy;

		const a = 4.0*g_dy*g_dy/dt;
		const aaAb = 4.0+a*a;
		for (let j=1; j<nny-1; j++) {
			// set b[], u[1],ph[1][]
			for (let i=1; i<nnx-1; i++) {
				g_bRe[i] = 2.0*ph[i][j][0]-a*ph[i][j][1] - ph[i+1][j][0] - ph[i-1][j][0];
				g_bIm[i] = 2.0*ph[i][j][1]+a*ph[i][j][0] - ph[i+1][j][1] - ph[i-1][j][1];
			}
			g_uRe[1] = -2.0/aaAb;
			g_uIm[1] = -a/aaAb;
			ph[1][j][0] = g_bRe[1]*g_uRe[1] - g_bIm[1]*g_uIm[1];
			ph[1][j][1] = g_bIm[1]*g_uRe[1] + g_bRe[1]*g_uIm[1];
			//forward elimination
			for (let i=2; i<nnx-1; i++) { //forward elimination
				const auAb = (-2.0-g_uRe[i-1])*(-2.0-g_uRe[i-1])+(a-g_uIm[i-1])*(a-g_uIm[i-1]);
				g_uRe[i] = (-2.0-g_uRe[i-1])/auAb;
				g_uIm[i] = -(a-g_uIm[i-1])/auAb;
				ph[i][j][0] = (g_bRe[i]-ph[i-1][j][0])*g_uRe[i] - (g_bIm[i]-ph[i-1][j][1])*g_uIm[i];
				ph[i][j][1] = (g_bRe[i]-ph[i-1][j][0])*g_uIm[i] + (g_bIm[i]-ph[i-1][j][1])*g_uRe[i];
			}
			//backward substitution
			for (let i=nnx-3; i>=1; i--) {
				ph[i][j][0] -= ph[i+1][j][0]*g_uRe[i] - ph[i+1][j][1]*g_uIm[i];
				ph[i][j][1] -= ph[i+1][j][0]*g_uIm[i] + ph[i+1][j][1]*g_uRe[i];
			}
		}
	}

	function kyStep( ph, dt ) {
		const nnx=g_NNx, nny=g_NNy;

		const a = 4.0*g_dy*g_dy/dt;
		const aaAb = 4.0+a*a;
		for (let i=1; i<nnx-1; i++) {
			// set b[], u[1],ph[][1]
			for (let j=1; j<nny-1; j++) {
				g_bRe[j] = 2.0*ph[i][j][0]-a*ph[i][j][1] - ph[i][j+1][0] - ph[i][j-1][0];
				g_bIm[j] = 2.0*ph[i][j][1]+a*ph[i][j][0] - ph[i][j+1][1] - ph[i][j-1][1];
			}
			g_uRe[1] = -2.0/aaAb;
			g_uIm[1] = -a/aaAb;
			ph[i][1][0] = g_bRe[1]*g_uRe[1] - g_bIm[1]*g_uIm[1];
			ph[i][1][1] = g_bIm[1]*g_uRe[1] + g_bRe[1]*g_uIm[1];
			//forward elimination
			for (let j=2; j<nny-1; j++) {
				const auAb = (-2.0-g_uRe[j-1])*(-2.0-g_uRe[j-1])+(a-g_uIm[j-1])*(a-g_uIm[j-1]);
				g_uRe[j] = (-2.0-g_uRe[j-1])/auAb;
				g_uIm[j] = -(a-g_uIm[j-1])/auAb;
				ph[i][j][0] = (g_bRe[j]-ph[i][j-1][0])*g_uRe[j] - (g_bIm[j]-ph[i][j-1][1])*g_uIm[j];
				ph[i][j][1] = (g_bRe[j]-ph[i][j-1][0])*g_uIm[j] + (g_bIm[j]-ph[i][j-1][1])*g_uRe[j];
			}
			//backward substitution
			for (let j=nny-3; j>=1; j--) {
				ph[i][j][0] -= ph[i][j+1][0]*g_uRe[j] - ph[i][j+1][1]*g_uIm[j];
				ph[i][j][1] -= ph[i][j+1][0]*g_uIm[j] + ph[i][j+1][1]*g_uRe[j];
			}
		}
	}

	// steepest descent method: psi_next = |psi) - damp*|psi)(psi|H-E|psi)
	// H = d^2/dx^2 + d^2/dy^2 + V
	function lossEnergy( ph, vv, damp ) {
		const nnx=g_NNx, nny=g_NNy, h2 = 2.0*g_dx*g_dx, ee = meanKinetic(ph) + meanPotential(ph,vv);

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_wrk[i][j][0] = -(ph[i+1][j][0]+ph[i-1][j][0]+ph[i][j+1][0]+ph[i][j-1][0]
								-4.0*ph[i][j][0])/h2+(g_vv[i][j]-ee)*ph[i][j][0];
				g_wrk[i][j][1] = -(ph[i+1][j][1]+ph[i-1][j][1]+ph[i][j+1][1]+ph[i][j-1][1]
								-4.0*ph[i][j][1])/h2+(g_vv[i][j]-ee)*ph[i][j][1];
			}
		}
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				ph[i][j][0] -= damp*g_wrk[i][j][0];
				ph[i][j][1] -= damp*g_wrk[i][j][1];
			}
		}
		normalize(ph);
	}


	// --------------------  utility  --------------------

	function norm( ph ) {
		const nnx=g_NNx, nny=g_NNy;

		let p = 0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				p += (ph[i][j][0]*ph[i][j][0]+ph[i][j][1]*ph[i][j][1]);
			}
		}
		return p*g_dx*g_dy;
	}

	function normalize( ph ) {
		const nnx=g_NNx, nny=g_NNy;

		const a = 1.0/Math.sqrt(norm(ph));
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				ph[i][j][0] *= a;
				ph[i][j][1] *= a;
			}
		}
	}

	function meanPotential( ph, vv ) {
		const nnx=g_NNx, nny=g_NNy;

		let p = 0.0
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				p += vv[i][j]*(ph[i][j][0]*ph[i][j][0]+ph[i][j][1]*ph[i][j][1]);
			}
		}
		return p*g_dx*g_dy;
	}

	function meanKinetic( ph ) {
		const nnx=g_NNx, nny=g_NNy, h2=g_dx*g_dx;
		let p = 0.0;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const d2phRe = (ph[i+1][j][0]+ph[i-1][j][0]+ph[i][j+1][0]+ph[i][j-1][0]-4.0*ph[i][j][0])/h2;
				const d2phIm = (ph[i+1][j][1]+ph[i-1][j][1]+ph[i][j+1][1]+ph[i][j-1][1]-4.0*ph[i][j][1])/h2;
				p += (ph[i][j][0]*d2phRe+ph[i][j][1]*d2phIm);
			}
		}
		return -0.5*p*g_dx*g_dy;
	}

	function psiDensity(i,j) {
		return (g_psi[i][j][0]*g_psi[i][j][0]+g_psi[i][j][1]*g_psi[i][j][1]);
	}

	function psiPhase(i,j) { // 0...2PI
		return (Math.PI + Math.atan2(g_psi[i][j][1],g_psi[i][j][0])); // atan2(y,x)
	}

	function psiXCurrent(i,j) {
		const pRe = (g_psi[i+1][j][1]-g_psi[i-1][j][1])/(2*g_dx);
		const pIm = (-g_psi[i+1][j][0]+g_psi[i-1][j][0])/(2*g_dx);

		return (g_psi[i][j][0]*pRe + g_psi[i][j][1]*pIm)*g_dx*g_dy;
	}

	function psiYCurrent(i,j) {
		const pRe = (g_psi[i][j+1][1]-g_psi[i][j-1][1])/(2*g_dy);
		const pIm = (-g_psi[i][j+1][0]+g_psi[i][j-1][0])/(2*g_dy);

		return (g_psi[i][j][0]*pRe + g_psi[i][j][1]*pIm)*g_dx*g_dy;
	}

	function setCloud() {
		const nnx=g_NNx, nny=g_NNy;

		set_srnd();

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_cloud[i][j] = 0;
			}
		}
		let s = 0;
		let ip = 0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_cloud[i][j] = 0;
				s += psiDensity(i,j)*g_dx*g_dy;
				while (s>g_srnd[ip] && ip<1000) {
					g_cloud[i][j] += 1;
					ip += 1;
				}
			}
		}
	}

	function set_srnd() {

		g_srnd[0] = Math.random();
		for (let i=1; i<1001; i++) {
			g_srnd[i] = g_srnd[i-1] + Math.random();
		}
		for (let i=0; i<1000; i++) {
			g_srnd[i] = g_srnd[i]/g_srnd[1000];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme )
		evolve:			timeEvolution,			// timeEvolution( lossSW )

		setCloud:		setCloud,				// setCloud()

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_NNy, g_dx, g_timeStep ]; },
		getNow:			function() { return [ g_sysTime, norm(g_psi), meanKinetic(g_psi), meanPotential(g_psi,g_vv) ]; },
		getPsiDensity:	psiDensity,				// psiDensity(i,j)
		getPsiPhase:	psiPhase,				// psiPhase(i,j)
		getPsiCurrent:	function(i,j) { return [ psiXCurrent(i,j), psiYCurrent(i,j) ]; },
		getPsi:			function(i,j) { return [ g_psi[i][j][0], g_psi[i][j][1] ]; },
		getVext:		function(i,j) { return g_vv[i][j]; },
		getCloud:		function(i,j) { return g_cloud[i][j]; },
	};

})(); // ====================  end of wavePacketQED2D Module  ====================


const js005 = (function(){ // ====================  js Module  ====================

	const theModule = wavePacketQED2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');
	let imgVext = null;			// image in draw()

	let v_theme = 1;			// 0:free space, 1:parabolic potential, 2:tunnel effect 3:step hill 4:step higher hill
	let v_lossSW = 0;			// if lossSW==1, call lossEnergy()

	let p_auLength, p_auTime, p_auEnergy, p_au2eV; // <-- theModule.getAUinSI();
	let p_NNx, p_NNy, p_dx, p_timeStep; // <-- theModule.getSysParam();
	let p_NNz;
	let sysTime, norm, meanKinetic, meanPotential;

	let dispMode = 1;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let onlyFlag = false;
  let breakFlag = false;

	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics - mouse event listener
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme ); // ( nn, BoxSizeInNM, contTemp )
			imgVext = null;
			[ p_auLength, p_auTime, p_auEnergy, p_au2eV ] = theModule.getAUinSI();
			[ p_NNx, p_NNy, p_dx, p_timeStep ] = theModule.getSysParam();
			p_NNz = p_NNx;
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift )
			g3d.init( p_NNx, p_NNy, p_NNz, p_dx, xCanvasSize, yCanvasSize, 300, 20 );
			g3d.drawVectorField2D.threshold = 0.2;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_lossSW );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			onlyFlag = true;
			theModule.evolve( v_lossSW );
		}

		draw( ctx, dispMode );

		requestAnimationFrame(animate);
	}


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const xp = 80, yp = 20, sc = 2, xBoxSize = 320, yBoxSise = 320, yTextPos = yCanvasSize-45; // image2d
		const colour = { text:'#888888', box:'#999900', vext:'#00ff00' };

		ctx.clearRect( 0, 0, xCanvasSize, yCanvasSize );

		if ( dispMode<=3 ) {
			ctx.strokeStyle = colour.box;
			ctx.strokeRect( xp, yp, xBoxSize, yBoxSise );

			if ( imgVext == null ) setImg_Vext( ctx, p_NNx, p_NNy, xp, yp, sc );
			ctx.putImageData( imgVext, xp, yp ); // potential vv[][] imageData
		}

		if ( dispMode==0 || dispMode==1 || dispMode==2 ) { // 0:density 1:phase 2::prob.current
			drawPh( ctx, dispMode, p_NNx, p_NNy, xp, yp, sc );
		} else if ( dispMode==3 ) { // 3:cloud
			drawCloud( ctx, p_NNx, p_NNy, xp, yp, sc );

		} else if (dispMode==4 ) { // 4: 3D Vext(x,y)
			const zFunc = function(i,j) { return 10.0*theModule.getVext(i,j); };
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, "#00aa00", 2 );
		} else if (dispMode==5 ) { // 4: 3D density(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j); };
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, 1.0, 2 );
		} else if (dispMode==6 ) { // 4: 3D Re[psi](x,y)
			const zFunc = function(i,j) { return 150.0*theModule.getPsi(i,j)[0]; };
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, 1.0, 1 );
		} else if (dispMode==7 ) { // 7: 3D phase(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j)};
			const colorFunc = function(i,j) {
				const hue = theModule.getPsiPhase(i,j)*180.0/Math.PI;
				const z = 1500.0*theModule.getPsiDensity(i,j);
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? "hsl("+hue+",100%,50%)" : "#444444";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );
		} else if (dispMode==8 ) { // 8: 3D (density+Vext)(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j) + 10.0*theModule.getVext(i,j) };
			const colorFunc = function(i,j) {
				const zc = 1500.0*theModule.getPsiDensity(i,j)
				const th = (18120 - Math.floor(180.0*zc/g3d.cz0))%360;
				return ( Math.abs(zc/g3d.cz0)>0.005 ) ? "hsl("+(th)+",100%,50%)" : "hsl("+(th)+",30%,20%)";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );
		} else if (dispMode==9 ) { // 9: 3D (phase+Vext)(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j) + 10.0*theModule.getVext(i,j) };
			const colorFunc = function(i,j) {
				const hue = theModule.getPsiPhase(i,j)*180.0/Math.PI;
				const z = 1500.0*theModule.getPsiDensity(i,j);
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? "hsl("+(hue)+",100%,50%)" : "#004400";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );
		} else if (dispMode==10 ) { // 10: 3D (density+Vext)(x,y) + prob.current
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j) + 10.0*theModule.getVext(i,j) };
			const colorFunc = function(i,j) {
				const zc = 1500.0*theModule.getPsiDensity(i,j)
				const th = (18120 - Math.floor(180.0*zc/g3d.cz0))%360;
				return ( Math.abs(zc/g3d.cz0)>0.005 ) ? "hsl("+(th)+",100%,50%)" : "hsl("+(th)+",30%,20%)";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );

			const vFunc = function(i,j) {
				const mag = 15000.0;
				const z = 1500.0*theModule.getPsiDensity(i,j)
				if ( z<0.01 ) return [ 0, 0 ];
				let cx,cy;
				[ cx, cy ] = theModule.getPsiCurrent(i,j);
				return [ cx*mag, cy*mag ];
			}
			//drawVectorField2D(ctx, rotAngle, zFunc, vFunc, colorMode, inc [, showBox] )
			g3d.drawVectorField2D(ctx, 0.0, zFunc, vFunc, 1, 4, 2 )
		}

		[ sysTime, norm, meanKinetic, meanPotential ] = theModule.getNow();
		const totalEnergy = meanKinetic + meanPotential;
		ctx.fillStyle = colour.text;
		ctx.fillText(`time = ${sysTime} (au)`, 20, yTextPos);
		ctx.fillText(`norm = ${norm.toFixed(15)}`, 240, yTextPos);
		ctx.fillText(`kineticE = ${(meanKinetic*p_au2eV).toFixed(2)} (eV)`, 20, yTextPos+20);
		ctx.fillText(`potentialE = ${(meanPotential*p_au2eV).toFixed(2)} (eV)`, 240, yTextPos+20);
		ctx.fillText(`totalE= ${(totalEnergy*p_au2eV).toFixed(2)} (eV)`, 20, yTextPos+40);
		ctx.fillText("box = 80x80 (au)", 240, yTextPos+40);

	}

	function setImg_Vext( ctx, nnx, nny, xp, yp, sc ) {
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const vij = theModule.getVext(i,j);
				const g = Math.min(Math.floor(vij*50),255);
				ctx.fillStyle = "rgb(0,"+g+",0)";
				ctx.fillRect( i*sc+xp, (nny-j-1)*sc+yp, sc, sc );
			}
		}
		imgVext = ctx.getImageData(xp,yp,320,320);
	}

	function drawPh( ctx, drawMode, nnx, nny, xp, yp, sc ) {
		const velocityMag = 300000.0;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const d = theModule.getPsiDensity(i,j)*100000;
				const vij = theModule.getVext(i,j);
				const g = Math.min(Math.floor(vij*50),255);
				if (d>20.0) {
					if (drawMode==0 ||drawMode==2) { // 0:density 2:prob.current
						const a = Math.min(Math.floor(d),255);
						ctx.fillStyle = `rgb(${a},${g},${a})`;
					} else if (drawMode==1) { // 1:phase
						const p = theModule.getPsiPhase(i,j)*180.0/Math.PI;
						const a = 0.2*Math.min(Math.floor(d),255);
						ctx.fillStyle = `hsl(${p},100%,${a}%)`;
					}
					ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp,sc,sc);
				}
			}
		}
		if (drawMode==2) { // 2:prob.current, draw current lines
			for (let i=1; i<nnx-1; i+=4) {
				for (let j=1; j<nny-1; j+=4) {
					const d = theModule.getPsiDensity(i,j)*100000;
					if (d>20.0) {
						let cxij, cyij;
						[ cxij, cyij ] = theModule.getPsiCurrent(i,j)
						const cx = cxij*p_timeStep*velocityMag;
						const cy = -cyij*p_timeStep*velocityMag;
						const col = (cx>0) ? "rgb(80,80,250)" : "rgb(250,80,80)";
						drawLine( ctx, i*sc+xp, (nny-j-1)*sc+yp, i*sc+cx+xp, (nny-j-1)*sc+cy+yp, col );
					}
				}
			}
		}
	}

	function drawCloud( ctx, nnx, nny, xp, yp, sc ) { // drawMode = 3:cloud

		if ( !pauseFlag || onlyFlag ) {
			theModule.setCloud();
			onlyFlag = false;
		}
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const cij = theModule.getCloud(i,j)
				if ( cij>0 ) {
					const vij = theModule.getVext(i,j);
					const g = Math.min(Math.floor(vij*50),255);
					const p = Math.min( cij, 5 ) ;
					ctx.fillStyle = "hsl("+(75-p*15)+",100%,50%)";
					ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp,sc,sc);
				}
			}
		}
	}

	function drawLine( ctx, x1, y1, x2, y2, color ) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------

	// g3d_extension grid2d  created 2023.06.01, last updated 2023.06.04
	// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
	g3d.drawGrid2D = function ( ctx, rotAngle, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawGrid2D.threshold = 0.005;

	// g3d_extension gvextor field  created 2023.06.01, last updated 2023.06.08
	// g3d.drawVectorField2D(ctx, rotAngle, zFunc, vFunc, colorMode, inc [, showBox] )
	g3d.drawVectorField2D = function(ctx, rotAngle, zFunc, vFunc, colorMode, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, threshold = g3d.drawVectorField2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii; if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj+=inc) {
				let j=jj; if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;

				const f = zFunc(i,nny-j-1);
				const x = i*dx, y = j*dx, z = f + g3d.cz0;
				let vx, vy;
				[ vx, vy ] = vFunc(i,nny-j-1);
				if ( vx*vx+vy*vy < threshold ) continue;
				const x1 = x + vx, y1 = y -vy, z1 = z;
				let colr;
				if ( colorMode==0 ) {
					const hue = Math.floor((Math.atan2(vy,vx))*180.0/Math.PI+180.0);
					colr = `hsl(${hue},100%,50%)`;
				} else if ( colorMode==1 ) {
					colr = (vx>0) ? "#3333ff" : "#ff3333";
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawVectorField2D.threshold = 0.5;


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setLossSW() {  // select lossSW
		v_lossSW = 0 + document.getElementById("slct_loss").selectedIndex;
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle( 15, -10 );
		g3d.zoom = 1.0;
	}

	function breakLoop() {
		breakFlag = true;
	}

  function getData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, norm, meanKinetic, meanPotential ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setLossSW:		setLossSW,		// setLossSW()
		setDispMode:	setDispMode,	// setDispMode()
		viewHome:		viewHome,		// viewHome()

    breakLoop:	breakLoop,	// breakLoop();
		getData:    getData,  // getData( pyMsg ) : return [ sysTime, norm, meanKinetic, meanPotential ]
	};

})(); // ====================  js005 module end  ====================


const js = js005;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js005] electron wave packet QED2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>in free box</option><option selected>in parabollic V=k r^2</option>
<option>tunnel effect</option><option>step hill</option>
<option>double slit</option><option>cylinder scattering</option>
</select>
    <span style="margin-right: 10px;"></span>
<label>loss:</label>
<select id="slct_loss" onChange="js.setLossSW()">
<option selected>off</option><option>on</option></select>
    <span style="margin-right: 20px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>2D: psi density</option><option selected>2D: phase (angle:Im(psi)/Re(psi))</option>
<option>2D: probability current</option><option>2D electron cloud</option>
<option>grid2D: Vext(x,y)</option><option>grid2D: density(x,y)</option>
<option>grid2D: Re[psi(x,y)]</option><option>grid2D: phase(x,y)</option>
<option>grid2D: (density + Vext)(x,y)</option><option>grid2D: (phase + Vext)(x,y)</option>
<option>grid2D: prob.current +(density + Vext)(x,y)</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code, and python control

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
time.sleep(3)

# python control
for i in range(10):
  [ sysTime, norm, meanKinetic, meanPotential ] = eval_js( 'js.getData({})'.format(i) )
  energy = meanKinetic + meanPotential
  print(
    "i=",i,
    ",\t time =", '{:.2f}'.format(sysTime),
    "(au), norm =", '{:.16f}'.format(norm),
    ", total energy =", '{:.6g}'.format(energy), "(au)" )
  time.sleep(3)
#

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")
